# 온라인 의류 쇼핑 사이트 사이즈 추천 프로그램 개발

2018년에 연간 총 거래액 4500억을 돌파한 국내 최대 규모의 온라인 편집샵인 '무신사' 사이트에서 판매하는 옷들의 구매후기를 추출한다. 웹 주소: https://store.musinsa.com<br>
또한 '무신사'는 구매 후기는 물론 사이즈 옵션을 선택할 때 참고할 수 있는 사이즈 추천란을 제공한다. 사이즈 추천 란에는 구매자별 키, 몸무게, 성별, 구매한 사이즈, 그리고 사이즈에 대한 평가(적당함, 큼, 작음)들이 적혀있다.<br>
그런데 문제는 이 사이트가 가지고 있는 모든 정보를 보여주는 것이 아니라 한 번에 100개의 데이터를 제공한다는 것이다. 사이즈 옵션을 아무것도 선택하지 않았을 때는 사이즈에 관계없이 최신 순으로 100개를 보여준다. 이는 데이터 개수도 부족하고 사이즈별 편차도 심한 상황이었다. 하지만 사이즈 옵션을 클릭하면 선택한 사이즈를 구매한 사람들의 사이즈 추천 기록만 최대 100개가 나온다. 예를 들어 L사이즈를 클릭하면 L사이즈를 구매하고 사이즈 추천을 남긴 데이터 100개를 열람할 수 있는 것이다.<br>
따라서 우리는 **이 사이즈별 클릭을 옷의 주소와 사이즈 옵션 개수에 관계없이 자동으로 실행할 수 있도록 일반화하여 구현**했다. 이를 통해 한 옷당 최대 (선택할 수 있는 사이즈 옵션 개수) x 100 개의 데이터를 추출할 수 있었다. 아래는 selenium과 BeautifulSoup를 이용해 위의 내용을 구현한 코드이다.

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome(executable_path=r'C:/Users/rlaxo/PycharmProjects/SW_Study/chromedriver_win32/chromedriver.exe')
# 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다.
driver.implicitly_wait(3)

# 사이즈 추천을 받고 싶은 옷의 주소를 입력한다.('무신사'사이트 주소에 한정해서 구동)
main_adress = 'https://store.musinsa.com/app/product/detail/312174/0'
driver.get(main_adress)

# 해당 옷에서 선택할 수 있는 사이즈의 옵션들을 size_names 리스트에 저장한다.
size_list_raw = driver.find_element_by_xpath('//*[@id="option1"]')
size_list = size_list_raw.find_elements_by_tag_name('option')
size_names = [option.get_attribute("value") for option in size_list]
size_names.remove('')
end_num = len(size_names)
print(size_names)

# size_datas_list 리스트에 사이즈별 데이터를 담을 리스트를 넣어준다.(list in list 형식)
size_datas_list = []
for i in range(len(size_names)):
    size_datas_list.append([])

# BeautifulSoup를 이용해 추출한 데이터의 형식을 [성별, 키, 몸무게, 사이즈가 잘 맞는지 여부(적당함, 큼, 작음), 구매한 사이즈]로 나타내기 위해 필요없는 부분을 잘라내주는 함수 arrange.
def arrange(str_a):
    str_b = str_a.replace('\t','').replace('\n','').replace('[회원추천]','').replace('기준','').replace('Size 구매','').replace('(','').replace(')','').replace("'",'').replace('/',' ').split()
    return str_b

# 선택할 수 있는 사이즈 옵션을 차례대로 선택해  size_datas_list의 리스트를 채우는 함수 select_fill
def select_fill():
    for i in range(len(size_names)):
        element = driver.find_element_by_id('option1')
        element.send_keys(size_names[i][0])
        time.sleep(2) # 옵션을 선택하고 2초를 기다려준다.

        # arrdata는 list in list 형식으로 [성별, 키, 몸무게, 사이즈가 잘 맞는지 여부(적당함, 큼, 작음), 구매한 사이즈]형식의 리스트들을 하나의 list로 묶어놓음.
        content = driver.page_source
        soup = BeautifulSoup(content, 'html.parser')
        elements = soup.find('div',class_= 'option_size_recom').find_all('p',class_='size_content')
        arrdata = list(map(lambda x: arrange(x.text),elements))

        # size_datas_list의 리스트들을 채움. '적당함'의 경우 구매한 사이즈와 동일한 사이즈를 담는 리스트로 [키, 몸무게]형식의 리스트를 추가하고, '큼'의 경우 한 치수 작게, '작음'의 경우 한 치수 크게 보정함.
        for j in arrdata:
            if (j[3] == '적당함'):
                size_datas_list[i].append([int(j[1].replace('cm','')), int(j[2].replace('kg',''))])
            elif ((j[3] == '작음') & (i != end_num - 1)):
                size_datas_list[i + 1].append([int(j[1].replace('cm','')), int(j[2].replace('kg',''))])
            elif ((j[3] == '큼') & (i != 0)):
                size_datas_list[i - 1].append([int(j[1].replace('cm','')), int(j[2].replace('kg',''))])

# 같은 회사거나 비슷한 사이즈의 옷의 주소를 입력해 데이터 추가하는 함수 more_adresss
def more_adresss(new_adress):
    driver.get(new_adress)

    size_list_raw = driver.find_element_by_xpath('//*[@id="option1"]')
    size_list = size_list_raw.find_elements_by_tag_name('option')
    size_names = [option.get_attribute("value") for option in size_list]
    size_names.remove('')

    if (end_num != len(size_names)):
        print("이 주소에선 선택할 수 있는 사이즈 옵션 개수가 다릅니다!")
    else:
        select_fill()

select_fill()
more_adresss('https://store.musinsa.com/app/product/detail/435579/0')

adress_number = [1125023, 1125022, 1125021, 1125020, 1125019, 1125018, 1125017, 1125009, 1125008, 1125009, 1125007, 1125006, 1125005,
                 1125004, 1110030, 1110029, 1110028, 1110027, 1110026, 1107323]

adress = 'https://store.musinsa.com/app/product/detail/{}/0'
for i in adress_number:
    url = adress.format(str(i))
    more_adresss(url)
    

# print(len(size_datas_list))
# 데이터 출력
for i in range(len(size_datas_list)):
    if i == 0:
        for j in size_datas_list[i]:
            j.append('S')
    elif i ==1:
        for j in size_datas_list[i]:
            j.append('M')
    elif i == 2:
        for j in size_datas_list[i]:
            j.append('L')
    else:
        for j in size_datas_list[i]:
            j.append('XL')
            

new_list = []
for i in size_datas_list:
    for j in i:
        new_list.append(j)
        
print(new_list)

import csv

with open("output_size.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(new_list)


['S', 'M', 'L', 'XL']
[[174, 68, 'S'], [158, 50, 'S'], [159, 52, 'S'], [165, 56, 'S'], [160, 47, 'S'], [165, 55, 'S'], [183, 75, 'S'], [155, 41, 'S'], [164, 52, 'S'], [162, 58, 'S'], [155, 50, 'S'], [169, 57, 'S'], [162, 50, 'S'], [165, 50, 'S'], [163, 52, 'S'], [155, 50, 'S'], [165, 48, 'S'], [158, 50, 'S'], [168, 60, 'S'], [165, 53, 'S'], [157, 45, 'S'], [163, 58, 'S'], [152, 44, 'S'], [160, 50, 'S'], [162, 49, 'S'], [160, 50, 'S'], [166, 59, 'S'], [164, 56, 'S'], [160, 53, 'S'], [166, 50, 'S'], [158, 45, 'S'], [163, 53, 'S'], [157, 41, 'S'], [161, 50, 'S'], [152, 47, 'S'], [165, 52, 'S'], [163, 53, 'S'], [161, 50, 'S'], [158, 53, 'S'], [160, 55, 'S'], [167, 55, 'S'], [168, 53, 'S'], [153, 42, 'S'], [170, 70, 'S'], [165, 49, 'S'], [166, 60, 'S'], [163, 48, 'S'], [169, 66, 'S'], [163, 54, 'S'], [166, 50, 'S'], [163, 48, 'S'], [160, 53, 'S'], [153, 43, 'S'], [158, 48, 'S'], [153, 51, 'S'], [167, 48, 'S'], [165, 60, 'S'], [165, 50, 'S'], [163, 52, 'S'], [172, 60, 'S'], [150, 50, 'S'], [